In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np

import torch

from tqdm import tqdm
import time

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.cnt = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.cnt += n
        self.avg = self.sum / self.cnt


def accuracy(logits, labels):
    preds = torch.argmax(logits, dim=1)
    return torch.sum(preds == labels) / len(labels)


def eval_fn(model, loader, device):

    score = AverageMeter()
    model.eval()

    t = tqdm(loader)
    with torch.no_grad():  # no gradient needed
        for images, labels in t:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            acc = accuracy(outputs, labels)
            score.update(acc.item(), images.size(0))

            t.set_description('(=> Test) Score: {:.4f}'.format(score.avg))

    return score.avg

In [ ]:
def train_fn(model, optimizer, criterion, loader, device):
    
    time_begin = time.time()
    score = AverageMeter()
    losses = AverageMeter()
    model.train()
    time_train = 0

    t = tqdm(loader)
    for images, labels in t:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        acc = accuracy(logits, labels)
        n = images.size(0)
        losses.update(loss.item(), n)
        score.update(acc.item(), n)

        t.set_description('(=> Training) Loss: {:.4f}'.format(losses.avg))

    time_train += time.time() - time_begin
    print('training time: ' + str(time_train))
    return score.avg, losses.avg

In [7]:
def plot(k_folds, val_accuracies, train_accuracies, save_folder="plots"):
    # Create the save folder if it doesn't exist
    os.makedirs(save_folder, exist_ok=True)

    # Check if k_folds is 1, create a single plot without subplots
    if k_folds == 1:
        fig, ax = plt.subplots(figsize=(10, 5))
        val_scores = val_accuracies[0]
        train_scores = train_accuracies[0]

        # Plot the accuracy scores for each epoch
        epochs = np.arange(1, len(val_scores) + 1)

        # Plot validation accuracy
        ax.plot(epochs, val_scores, label='Validation')

        # Plot training accuracy
        ax.plot(epochs, train_scores, label='Training', linestyle='--')

        ax.set_title('Accuracy Scores')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy')
        ax.legend()

        # Save the plot
        save_path = os.path.join(save_folder, "accuracy_plot.png")
        plt.savefig(save_path)
        plt.show()

    else:
        # For k_folds greater than 1, create subplots
        fig, axes = plt.subplots(nrows=k_folds, ncols=1, figsize=(10, 5 * k_folds))

        # Plot accuracy scores for each fold
        for fold in range(k_folds):
            # Extract accuracy scores for the fold
            val_scores = val_accuracies[fold]
            train_scores = train_accuracies[fold]

            # Plot the accuracy scores for each epoch
            epochs = np.arange(1, len(val_scores) + 1)

            # Plot validation accuracy
            axes[fold].plot(epochs, val_scores, label=f'Validation')

            # Plot training accuracy
            axes[fold].plot(epochs, train_scores, label=f'Training', linestyle='--')

            axes[fold].set_title(f'Fold {fold + 1} Accuracy Scores')
            axes[fold].set_xlabel('Epoch')
            axes[fold].set_ylabel('Accuracy')
            axes[fold].legend()

        # Adjust layout and save the plot
        plt.tight_layout()
        save_path = os.path.join(save_folder, "accuracy_plots.png")
        plt.savefig(save_path)
        plt.show()